# Final Project, COMS 4995_005, Deep Learning

## Team Information

Team Member1 (Name,UNI): Richard Kennedy, rk2860

Python Version: 2
Utilizing Pytorch

In [2]:
import torch
import cPickle
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchtext.vocab as Vocab

import torchvision.datasets as dset
import torchvision.transforms as transforms

import PIL
import random
import numpy


In [3]:
cap = dset.CocoCaptions(root = './train2014',
                        annFile = 'annotations/captions_train2014.json',
                        transform= transforms.Compose([transforms.Resize([224, 224]), transforms.ToTensor()]))

res = models.resnet50(pretrained=True)
vdim = 100
rSize = 2048
x = Vocab.GloVe(name='6B', dim= vdim)

paddSize =  rSize - vdim



loading annotations into memory...
Done (t=3.37s)
creating index...
index created!


In [4]:
a = x.vectors[x.stoi['man']]
print a.size()
b = torch.FloatTensor(paddSize).zero_()
paddedVec = torch.cat((a, b), 0)

torch.Size([100])


In [5]:
print "HELLO"
# img, target = cap[0]
# words = ['StopLSTM'] + target[0].split(' ') + ['EndLSTM']
# print words


# print wordVec(Vocab[words[1]])

HELLO


## Part 1

In [26]:
class LSTM(nn.Module):
    def __init__(self, embed_size, hidden_size, res50):
        super(LSTM, self).__init__()
        
        self.Esize = embed_size
        self.Hsize = hidden_size
        self.Wix = nn.Linear(embed_size, hidden_size)
        self.Wim = nn.Linear(hidden_size, hidden_size)
        self.Wfx = nn.Linear(embed_size, hidden_size)
        self.Wfm = nn.Linear(hidden_size, hidden_size)
        self.Wox = nn.Linear(embed_size, hidden_size)
        self.Wom = nn.Linear(hidden_size, hidden_size)
        self.Wcx = nn.Linear(embed_size, hidden_size)
        self.Wcm = nn.Linear(hidden_size, hidden_size)
        
        self.Dense1 = nn.Linear(hidden_size, 1000)
        self.Dense2 = nn.Linear(1000, 25000)
        #self.Dense3 = nn.Linear(10000, 400000)
        #res50 = torchvision.models.resnet50(pretrained=True)
        self.res50NoGrad = torch.nn.Sequential(*list(res50.children())[:-2])
        #Every layer besides the last layer does not need to be trained
        for param in self.res50NoGrad.parameters():
            param.requires_grad = False
        
        #set embed_size to dimensions of self.res50Last
        #output should be second to last layer
        self.res50Last = torch.nn.Sequential(list(res50.children())[-2])
        
        
        #self.glove = Vocab.GloVe(name='840B', dim= embed_size)
        
        

    def forward(self, x, m = None, c = None, CNN = False ):
        
        if not CNN:
            #glove requires x to be one hot word encoding?
            #x = self.glove(x)
            i = F.sigmoid(self.Wix(x)+self.Wim(m))
            f = F.sigmoid(self.Wfx(x)+self.Wfm(m))
            o = F.sigmoid(self.Wox(x)+self.Wom(m))
            h = F.tanh(self.Wcx(x) + self.Wcm(m))
            newC = f*c + i*h
            newM = o*newC
            #prediction dimension should line up with vocab_size
            #fine tune hidden dimension to make that work
            
            #log_softmax might not be necessary -> due to backprop
            
            #calculations: hidden dimensions = 50, embed_size = 2048, vocab_size not used
            #prediction = F.log_softmax(newM,)
            
            pred1 = F.relu(self.Dense1(newM))
            prediction = F.relu(self.Dense2(pred1))
            #prediction = self.Dense3(pred2)
            prediction = F.log_softmax(prediction)
            
            return prediction, newC, newM
        
        else:
            x = self.res50NoGrad(x)
            x = self.res50Last(x)
            x = x.resize(1, self.Esize)
            i = F.sigmoid(self.Wix(x))
            f = F.sigmoid(self.Wfx(x))
            o = F.sigmoid(self.Wox(x))
            h = F.tanh(self.Wcx(x))
            c = i*h
            m = o*c
            
            return None, c, m

# net = LSTM(2048, 50, res)
# img, target = cap[0]
# img = img.unsqueeze(0)
# junk, c0, m0 =  net.forward(Variable(img), CNN = True)

# a = x.vectors[x.stoi['man']]
# b = torch.FloatTensor(1998).zero_()
# paddedVec = torch.cat((a, b), 0)

# pred, c0, m0 = net.forward(Variable(paddedVec), m = m0, c = c0)
# pred = pred.unsqueeze(0)

## Part 2

In [27]:
##Train


startW = 'start'
endW = 'stop'
## was lazy and used cat and dog as start and stop word. 
## Definetely going to have to change that
net = LSTM(rSize, vdim, res)
import torch.optim as optim

criterion = nn.NLLLoss()
optimizer = optim.SGD(filter(lambda p: p.requires_grad,net.parameters()), lr=0.001, momentum=.2)

counter = 0

for epoch in range(0, 1000):
    permute = np.random.permutation(len(cap))
    logLoss = 0.0
    numGoals = 0
    numGenerals = 0
    localMins = 0
    torch.save(net, str(epoch)+'LSTM.pt')
    for i in range(0, len(cap)):
        img, target = cap[permute[i]]
        words = [startW] + target[0].split(' ') + [endW]
        length = len(words)
        img = img.unsqueeze(0)
        junk, c0, m0 =  net.forward(Variable(img), CNN = True)
        fullPred = None
        fullAnsw = None
        flip = True
        
        
        for j in range(0, length-1):
            lossPlus = 0
            #Going to need larger word corpus
            if words[j].lower() in x.stoi and words[j+1].lower() in x.stoi:

                a = x.vectors[x.stoi[words[j].lower()]]
                b = torch.FloatTensor(paddSize).zero_()
                paddedVec = torch.cat((a, b), 0)
                pred, c0, m0 = net.forward(Variable(paddedVec), m = m0, c = c0)
                
                index = x.stoi[words[j+1].lower()]
                if index < 25000:
                    answer = torch.zeros([1]).type('torch.LongTensor')
                    answer[0] = index
                    
                    
                else:
                    continue
                #temp = torch.FloatTensor(400000).zero_()
                #temp[x.stoi[words[j+1].lower()]] = 1

#                 answer = x.vectors[x.stoi[words[j+1].lower()]]
#                 answer = answer.unsqueeze(0)

                
                
#                 #Trying to remove the Local Minimums
#                 dist1 = torch.dist(pred, Variable(answer)).data[0]
#                 dist2 = torch.dist(pred, Variable(x.vectors[x.stoi[random.choice(x.stoi.keys())]])).data[0]
#                 if abs(dist1 - dist2) <= .8 or dist1 >= dist2:
#                     #pred.data[0] = Variable(x.vectors[x.stoi[random.choice(x.stoi.keys())]]).data[0]
                    
#                     numGoals+=1
#                     lossPlus+= .35
                    
#                 if torch.dist(pred, Variable(x.vectors[x.stoi['goals_none']])).data[0] <= 4.0:
                    
#                     numGenerals+=1   
#                     #pred.data[0] = Variable(x.vectors[x.stoi[random.choice(x.stoi.keys())]]).data[0]
#                     lossPlus+=.5
                
# #                 if dist1 < 3.5 and dist1 > 2.5:
# #                     localMins+= 1
# #                     lossPlus += .25
               
                
                
                loss = criterion(pred, Variable(answer))
                #loss.data[0]+=lossPlus
                loss.backward(retain_graph = True)
        
                optimizer.step()

                logLoss += loss.data[0]
                #logLoss += lossPlus
                if counter % 100 == 99:    
                    print("EPOCH: "+str(epoch) + ", LOSS: " + str(logLoss/100))
                    print loss.data[0]
                    #print("goals_none: "+ str(numGoals) + ", generals: "+str(numGenerals))
#                     numGoals = 0
#                     numGenerals = 0
                    logLoss = 0.0
        
                if counter == 0:
                    print("EPOCH: "+str(epoch) + ", LOSS: " + str(logLoss))
                    
                counter+=1   

#                 if flip:
#                     fullPred = pred.unsqueeze(0)
#                     fullAnsw = answer.unsqueeze(0)
#                     flip = False
#                 else:
#                     answer = answer.unsqueeze(0)
#                     pred = pred.unsqueeze(0)
#                     fullPred = torch.cat((fullPred, pred), 0)
#                     fullAnsw = torch.cat((fullAnsw, answer), 0)
                    
                    


        
#         loss = criterion(fullPred, Variable(fullAnsw))
#         loss.data[0]+=lossPlus
#         loss.backward()
        
#         optimizer.step()

#         logLoss += loss.data[0]
#         if counter % 100 == 99:    
#             print("EPOCH: "+str(epoch) + ", LOSS: " + str(logLoss/100))
#             logLoss = 0.0
        
#         if counter == 0:
#             print("EPOCH: "+str(epoch) + ", LOSS: " + str(logLoss))

#         counter+=1   
    
    

/Users/Richard/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


EPOCH: 0, LOSS: 10.1395254135
EPOCH: 0, LOSS: 10.0036272049
9.34074115753
EPOCH: 0, LOSS: 9.20677968651
7.43363904953
EPOCH: 0, LOSS: 8.24207474828
11.1552495956
EPOCH: 0, LOSS: 6.79219589213
10.9621248245
EPOCH: 0, LOSS: 7.9854288898
10.4130058289
EPOCH: 0, LOSS: 7.47164356261
10.5925998688
EPOCH: 0, LOSS: 7.26797458172
10.8862047195
EPOCH: 0, LOSS: 9.79891041189
4.42850875854
EPOCH: 0, LOSS: 7.80174483359
9.21429634094
EPOCH: 0, LOSS: 7.30309289187
11.0460977554
EPOCH: 0, LOSS: 7.43882817358
5.0699801445
EPOCH: 0, LOSS: 6.78464717209
5.88101148605
EPOCH: 0, LOSS: 6.72407829106
11.1056604385
EPOCH: 0, LOSS: 7.41315808982
3.84374570847
EPOCH: 0, LOSS: 6.86710194349
0.890671491623
EPOCH: 0, LOSS: 6.4943426609
5.93923664093
EPOCH: 0, LOSS: 6.26645407259
6.22255086899
EPOCH: 0, LOSS: 6.43520167351
11.6732730865
EPOCH: 0, LOSS: 6.48453542054
1.97042393684
EPOCH: 0, LOSS: 7.35856286168
78.4253463745
EPOCH: 0, LOSS: 6.68931759
11.5995121002
EPOCH: 0, LOSS: 6.04572455883
11.6850376129
EPOCH: 

EPOCH: 0, LOSS: 8.15125870705
10.2404565811
EPOCH: 0, LOSS: 8.43091962099
2.25981259346
EPOCH: 0, LOSS: 8.46407219172
10.8293142319
EPOCH: 0, LOSS: 47.903093453
10.5809955597
EPOCH: 0, LOSS: 8.32775066018
10.4140377045
EPOCH: 0, LOSS: 8.44869211078
10.385430336
EPOCH: 0, LOSS: 9.12931621313
10.3748426437
EPOCH: 0, LOSS: 8.3285217005
10.6791200638
EPOCH: 0, LOSS: 8.69353141606
10.4651880264
EPOCH: 0, LOSS: 43636.2739758
10.3221311569
EPOCH: 0, LOSS: 7014.48194186
10.3225183487
EPOCH: 0, LOSS: 1037.21173392
10.4344177246
EPOCH: 0, LOSS: 30.8357066494
10.9031162262
EPOCH: 0, LOSS: 8.20691363633
10.8138151169
EPOCH: 0, LOSS: 17.0958448434
10.5049791336
EPOCH: 0, LOSS: 8.83190348506
2.70125889778
EPOCH: 0, LOSS: 8.66657783031
10.3165512085
EPOCH: 0, LOSS: 8.65772653699
10.5501241684
EPOCH: 0, LOSS: 8.21343978047
10.6054410934
EPOCH: 0, LOSS: 8885.22042447
10.329785347
EPOCH: 0, LOSS: 1107.2513007
7.95067834854
EPOCH: 0, LOSS: 24.6170675421
10.2484922409
EPOCH: 0, LOSS: 28.8730917001
2.20424

KeyboardInterrupt: 

## Extra Credit

In [ ]:
#torch.save(net, str(epoch)+'DenseLSTM.pt')




net = LSTM(rSize, vdim, res)

def createInverse(dic):
    new_dic ={}
    for key in dic:
        new_dic[dic[key]] = key
    
    return new_dic

invert = createInverse(x.stoi)

def closest(vec, n=1):
    """
    Find the closest words for a given vector
    """
#     all_dists = [(w,torch.dist(vec, Variable(x.vectors[x.stoi[w]]))) for w in x.stoi]
#     return sorted(all_dists, key=lambda t: t[1].data.numpy()[0])[:n]
    
    maximum = 0
    maxIndex = 0
    for i in range(0, len(vec)):
        if vec[i] > maximum:
            maxIndex = i
            maximum = vec[i]
    maxIndex2 = 0
    maximum = 0
    for i in range(0, len(vec)):
        if vec[i] > maximum and i != maxIndex:
            maxIndex2 = i
            maximum = vec[i]
    
    return maxIndex, maxIndex2

def whenNotSorted(vec):
    before = vec[0]
    for i in range(1, len(vec)):
        if before >= vec[i]:
            before = vec[i]
        else:
            return i
    return -1

img, target = cap[10]
print target
img = img.unsqueeze(0)
junk, c0, m0 =  net.forward(Variable(img), CNN = True)
a = x.vectors[x.stoi['dog'.lower()]]
b = torch.FloatTensor(paddSize).zero_()
paddedVec = torch.cat((a, b), 0)
prediction, c0, m0 = net.forward(Variable(paddedVec), m = m0, c = c0)
print prediction.data[0][400]
print prediction.data[0][700]
strn, listVec =  closest(prediction.data[0], 1)

print whenNotSorted(prediction.data[0])
print invert[listVec]
newWord = invert[strn]
print newWord

print listVec, strn
k = input()
#print strn
while True:
    a = x.vectors[x.stoi[newWord.lower()]]
    b = torch.FloatTensor(paddSize).zero_()
    paddedVec = torch.cat((a, b), 0)
    prediction, c0, m0 = net.forward(Variable(paddedVec), m = m0, c = c0)
    strn, listVec =  closest(prediction.data[0], 1)
    print invert[listVec]
    newWord = invert[strn]
    print newWord
    #print strn
    
    

[u'A man riding skis down a snow covered slope.', u'Someone cross country skiing in their back yard', u'A person is skiing over a snowy hill.', u'A man is trying to ski down a small hill.', u'A person on skis in the snow with trees in back.']


/Users/Richard/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:56: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


-10.1388587952
-10.1388587952
7
the
the
0 0


In [174]:

print closest(prediction, 100)

print strn[0]


import random

print torch.dist(Variable(x.vectors[x.stoi[random.choice(x.stoi.keys())]]), Variable(x.vectors[x.stoi['goals_none']]))

print torch.dist(Variable(x.vectors[x.stoi['snow']]), Variable(x.vectors[x.stoi['ice']]))


[(u'goals_none', Variable containing:
 2.6643
[torch.FloatTensor of size 1]
), (u'65stk', Variable containing:
 2.7356
[torch.FloatTensor of size 1]
), (u'__________________________________', Variable containing:
 2.7474
[torch.FloatTensor of size 1]
), (u'prohertrib', Variable containing:
 2.7507
[torch.FloatTensor of size 1]
), (u'http://www.oklahomacitynationalmemorial.org', Variable containing:
 2.7676
[torch.FloatTensor of size 1]
), (u'tissottiming.com', Variable containing:
 2.8126
[torch.FloatTensor of size 1]
), (u'____________________________________________', Variable containing:
 2.8153
[torch.FloatTensor of size 1]
), (u'timewrn', Variable containing:
 2.8185
[torch.FloatTensor of size 1]
), (u'_____________________________________________', Variable containing:
 2.8302
[torch.FloatTensor of size 1]
), (u'www.slarmy.org', Variable containing:
 2.8431
[torch.FloatTensor of size 1]
), (u'http://www.co.mo.md.us', Variable containing:
 2.8616
[torch.FloatTensor of size 1]
), (

In [173]:
print torch.dist(Variable(x.vectors[x.stoi['hand']]), Variable(x.vectors[x.stoi['finger']]))

Variable containing:
 4.0605
[torch.FloatTensor of size 1]



In [196]:
print x.stoi['petunia']

101640
